In [ ]:
##by ltecho@whu.edu.cn
##
import pandas as pd
import numpy as np

In [ ]:
from pyecharts import Graph
ip_data_cluster = pd.read_csv('ip_file\\ip_data_friends.csv',delimiter=',',encoding='utf-8',dtype='str')
mid_nodes = []
mid_links = []
mid_nodes_set = set()
bind_domain_nodes = []
bind_domain_links = []
bind_domain_nodes_set = set()
attack_file_nodes = []
attack_file_links = []
attack_file_nodes_set = set()

for index,ip_data in ip_data_cluster.iterrows():
    #nodes.append({"name":ip_data.ip,"symbolSize":5})
    if str(ip_data.mid_friends_list)!="set()" and str(ip_data.mid_friends_list)!='nan':
        #print(ip_data.mid_friends_list)
        mid_friends_list = eval(ip_data.mid_friends_list)
        mid_nodes_set.add(ip_data.ip)
        for k in mid_friends_list:
            mid_nodes_set.add(k)
            mid_links.append({"source":ip_data.ip,"target":k})
    if str(ip_data.bind_domain_friends_list)!="set()" and str(ip_data.bind_domain_friends_list)!='nan':
        #print(ip_data.mid_friends_list)
        bind_domain_friends_list = eval(ip_data.bind_domain_friends_list)
        bind_domain_nodes_set.add(ip_data.ip)
        for k in bind_domain_friends_list:
            bind_domain_nodes_set.add(k)
            bind_domain_links.append({"source":ip_data.ip,"target":k})
    if str(ip_data.attack_file_friends_list)!="set()" and str(ip_data.attack_file_friends_list)!='nan':
        #print(ip_data.mid_friends_list)
        attack_file_friends_list = eval(ip_data.attack_file_friends_list)
        attack_file_nodes_set.add(ip_data.ip)
        for k in attack_file_friends_list:
            attack_file_nodes_set.add(k)
            attack_file_links.append({"source":ip_data.ip,"target":k})

            
for j in mid_nodes_set:
    mid_nodes.append({"name":j,"symbolSize":5})
    
for z in bind_domain_nodes_set:
    bind_domain_nodes.append({"name":z,"symbolSize":5})
for z in attack_file_nodes_set:
    attack_file_nodes.append({"name":z,"symbolSize":5})


In [ ]:
    
print('终端指纹ip节点数：'+str(len(mid_nodes)))
print('终端指纹ip关系数：'+str(len(mid_links)))
print('恶意样本ip节点数：'+str(len(attack_file_nodes)))
print('恶意样本ip关系数：'+str(len(attack_file_links)))
print('绑定域名ip节点数：'+str(len(bind_domain_nodes)))
print('绑定域名ip关系数：'+str(len(bind_domain_links)))

In [ ]:
len(attack_file_nodes)

In [ ]:
mid_graph = Graph("终端指纹力导图",width = 1800, height=1400)
mid_graph.add(
    "",
    mid_nodes,
    mid_links,
    graph_layout = "force",
    label_pos = "right",
    graph_repulsion = 10,
    line_curve = 0.2,
)

In [ ]:
mid_graph.render('ip_file\\终端指纹力导图.html')

In [ ]:
bind_domain_graph = Graph("绑定域名力导图",width = 1800, height=1400)
bind_domain_graph.add(
    "",
    bind_domain_nodes,
    bind_domain_links,
    graph_layout = "force",
    label_pos = "right",
    graph_repulsion = 10,
    line_curve = 0.2,
)

In [ ]:
bind_domain_graph.render('ip_file\\绑定域名力导图.html')

In [ ]:
attack_file_graph = Graph("恶意样本力导图",width = 1800, height=1400)
attack_file_graph.add(
    "",
    attack_file_nodes,
    attack_file_links,
    graph_layout = "force",
    label_pos = "right",
    graph_repulsion = 10,
    line_curve = 0.2,
)

In [ ]:
attack_file_graph.render('ip_file\\恶意样本力导图.html')

In [ ]:
#结合三种关联关系分析，进一步得到攻击者和攻击机构的关系网
#得到恶意样本控制中心IP列表
attack_file_df = pd.DataFrame(attack_file_links)
attacker_file_df_counts = attack_file_df.groupby('source').count()
attacker_file_df_counts = attacker_file_df_counts.reset_index()
attacker_file_df_counts.rename(columns={'target':'target_number'},inplace=True)
#终端指纹关系列表
mid_df = pd.DataFrame(mid_links)
#得到终端指纹关系中，源ip为恶意样本控制中心的关系数据
mid_attack_file_df = pd.merge(mid_df,attacker_file_df_counts,on='source',how='left')
attacker_file_df_counts['target'] = attacker_file_df_counts['source'].apply(lambda x:x)
mid_attack_file_df.dropna(axis=0,how='any',inplace=True)
#得到终端关系中，目标ip为恶意样本控制中心的关系数据
attack_file_df_target = attacker_file_df_counts[['target','target_number']]
mid_attack_file_df2 = pd.merge(mid_df,attack_file_df_target,on='target',how='left')
mid_attack_file_df2.dropna(axis=0,how='any',inplace=True)
#拼接两份关系数据，得到终端指纹和恶意样本关系的关联数据
mid_attack_file_frames = [mid_attack_file_df,mid_attack_file_df2]
mid_attack_file_data = pd.concat(mid_attack_file_frames,keys=['s','t'])

In [ ]:
##可视化查看
attacker_file_df_counts.sort_values(by='source')
mid_attack_file_data.query('source =="120.200.167.93"').sort_values(by='target')